# Setup

Clone YOLOv5 repo, install dependencies and check PyTorch and GPU.

In [12]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -r requirements.txt wandb

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup completed. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup completed. Using torch 1.8.1 _CudaDeviceProperties(name='GeForce MX250', major=6, minor=1, total_memory=2002MB, multi_processor_count=3)


# Prepare dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip -q "/content/drive/MyDrive/Thesis/dataset/GRanD_yolo_dataset.zip" -d "/content"

In [ ]:
!unzip -q "/content/drive/MyDrive/Thesis/dataset/without_dam_yolo_dataset.zip" -d "/content" 
!cp -r /content/without_dam_yolo_dataset/test /content/GRanD_yolo_dataset
!cp -r /content/without_dam_yolo_dataset/train /content/GRanD_yolo_dataset
!rm -r /content/without_dam_yolo_dataset

In [25]:
!pwd

/home/hue/Data/codes/carvansara/object_detection/yolov5


In [26]:
dataset_yaml_txt= """
# train and val data 
train: /home/hue/Data/codes/carvansara/dataset/carvansara_yolo/train
val: /home/hue/Data/codes/carvansara/dataset/carvansara_yolo/test

# number of classes
nc: 1

# class names
names: ['carvansara',]
"""

with open("data/carvansara.yaml", "w") as f:
    f.write(dataset_yaml_txt)

In [22]:
carvansara_yaml_path = '/home/hue/Data/codes/carvansara/object_detection/yolov5/data/carvansara.yaml'

## Weights & Biases Logging




In [10]:
# !pip install -q wandb
import wandb
wandb.login(host="http://localhost:8083")
# clear_output()
print("Weights & Biases Setup completed.")

Weights & Biases Setup completed.


# Train the models

In [18]:
!python train.py --img 400 --batch 10 --epochs 1  --single-cls --name "yolov5s_" --data carvansara.yaml  --weights yolov5s.pt

github: up to date with https://github.com/ultralytics/yolov5 ✅
Traceback (most recent call last):
  File "train.py", line 514, in <module>
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
  File "/home/hue/Data/codes/carvansara/object_detection/yolov5/yolov5/utils/general.py", line 151, in check_file
    assert len(files), f'File Not Found: {file}'  # assert file was found
AssertionError: File Not Found: carvansara_yaml_path


In [ ]:

hyp = """
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.00  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 75.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
"""
with open("/content/yolov5/data/n_hyp.scratch.yaml", "w") as f:
    f.write(hyp)

In [ ]:
!python train.py --img 448 --batch 150 --epochs 50  --single-cls --hyp "data/n_hyp.scratch.yaml" --name "s_wdam_s448e50singlecls_iou0_" --data GRanD.yaml --log-imgs 100 --weights yolov5s.pt

Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)

Namespace(adam=False, batch_size=150, bucket='', cache_images=False, cfg='', data='./data/GRanD.yaml', device='', epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/n_hyp.scratch.yaml', image_weights=False, img_size=[448, 448], local_rank=-1, log_imgs=100, multi_scale=False, name='s_wdam_s448e50singlecls_iou0_', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/s_wdam_s448e50singlecls_iou0_', single_cls=True, sync_bn=False, total_batch_size=150, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2020-11-27 00:39:04.301149: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'w

### 4. YOLOv5 size 640 epochs 10 + 15 singlecls

In [ ]:
# Train YOLOv5 640 10 singlecls
!python train.py --img 640 --batch 25 --epochs 25 --single-cls --name "s640e25singlecls_" --data GRanD.yaml --log-imgs 100 --weights yolov5l.pt

### 7. YOLOv5 size 448 epochs 10 + 15 rect single-cls

In [ ]:
# Train YOLOv5 448 10 rect single-cls
!python train.py --img 448 --batch 50 --epochs 50 --rect --single-cls --name "x_wdam_s448e50singleclsrect_" --data GRanD.yaml --log-imgs 100 --weights yolov5x.pt

Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)

Namespace(adam=False, batch_size=50, bucket='', cache_images=False, cfg='', data='./data/GRanD.yaml', device='', epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[448, 448], local_rank=-1, log_imgs=100, multi_scale=False, name='x_wdam_s448e50singleclsrect_', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=True, resume=False, save_dir='runs/train/x_wdam_s448e50singleclsrect_4', single_cls=True, sync_bn=False, total_batch_size=50, weights='yolov5x.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2020-11-26 07:36:38.417044: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_

### 8. YOLOv5 size 640 epochs 10 + 15 rect single-cls

In [ ]:
# Train YOLOv5 
!python train.py --img 640 --batch 25 --epochs 25 --rect --single-cls --name "s640e25singleclsrect_" --data GRanD.yaml --log-imgs 100 --weights yolov5l.pt

# test model

In [ ]:

!python detect.py --weights "/content/yolov5/runs/train/s448e50singleclsrect_/weights/best.pt" --img 640 --conf 0.5 --source "/content/GRanD_yolo_dataset/test/images"
#clear_output()

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.5, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/content/GRanD_yolo_dataset/test/images', update=False, view_img=False, weights=['/content/yolov5/runs/train/s448e50singleclsrect_/weights/best.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)

Fusing layers... 
Model Summary: 400 layers, 47364662 parameters, 0 gradients
image 1/543 /content/GRanD_yolo_dataset/test/images/0032.png: 352x640 1 items, Done. (0.027s)
image 2/543 /content/GRanD_yolo_dataset/test/images/0033.png: 352x640 1 items, Done. (0.026s)
image 3/543 /content/GRanD_yolo_dataset/test/images/0034.png: 352x640 1 items, Done. (0.026s)
image 4/543 /content/GRanD_yolo_dataset/test/images/0036.png: 352x640 1 items, Done. (0.026s)
image 5/543 /content/GRanD_yolo_dataset/test/images/0044.png: 352x640 1 items, Done. (0.025s)
image 6/543 /content/GRanD_yolo_dataset/te

In [ ]:
Image(filename='/content/yolov5/runs/detect/exp2/0019.png', width=800)

In [ ]:
!zip -r /content/runs_output.zip /content/yolov5/runs

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/ (stored 0%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/weights/ (stored 0%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/weights/best.pt (deflated 8%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/weights/last.pt (deflated 8%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/labels.png (deflated 3%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/test_batch2_pred.jpg (deflated 6%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/train_batch0.jpg (deflated 4%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/test_batch1_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/events.out.tfevents.1606309313.d3b298b93e96.893.0 (deflated 70%)
  adding: content/yolov5/runs/train/s448e50singleclsrect_/precision_recall_curve.png (deflated 26%)

In [ ]:
from google.colab import files
files.download("/content/runs_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>